In [1]:
from sklearn.datasets import load_wine
data = load_wine()
data.target[[10, 80, 140]]

#list(data.target_names)

array([0, 1, 2])

In [4]:
type(data)

sklearn.utils.Bunch

In [9]:
len(data.DESCR)

3479

In [11]:
abc = [1,2,3]
for i in range(len(abc)):
    if i!=len(abc):
        print(abc[i])

1
2
3


In [2]:
data.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

In [14]:
X = data.data
y = data.target

In [24]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [16]:
X_train.shape

(119, 13)

In [18]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, X_train,y_train, cv=10)


array([0.92307692, 0.92307692, 0.92307692, 0.83333333, 0.91666667,
       0.91666667, 0.91666667, 1.        , 0.90909091, 1.        ])

In [25]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 3), random_state=1)

cross_val_score(clf, X_train,y_train, cv=10)


array([0.38461538, 0.38461538, 0.38461538, 0.41666667, 0.41666667,
       0.41666667, 0.41666667, 0.36363636, 0.36363636, 0.4       ])

In [22]:
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
scaler = StandardScaler()  # doctest: +SKIP
# Don't cheat - fit only on training data
scaler.fit(X_train)  # doctest: +SKIP
X_train = scaler.transform(X_train)  # doctest: +SKIP
# apply same transformation to test data
X_test = scaler.transform(X_test)  # doctest: +SKIP

In [23]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 3), random_state=1)

cross_val_score(clf, X_train,y_train, cv=10)


array([0.92307692, 1.        , 1.        , 1.        , 1.        ,
       0.91666667, 1.        , 1.        , 1.        , 1.        ])

In [26]:
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
X, y = make_classification(random_state=0)

clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)

# clf.predict(X_test[:2])

# clf.score(X_test, y_test)
cross_val_score(clf, X_train,y_train, cv=10)


array([1.        , 1.        , 0.92307692, 0.75      , 0.91666667,
       0.91666667, 0.91666667, 1.        , 1.        , 1.        ])

In [27]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X, y)

cross_val_score(clf, X_train,y_train, cv=10)


C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

array([0.38461538, 0.38461538, 0.38461538, 0.5       , 0.5       ,
       0.41666667, 0.5       , 0.45454545, 0.36363636, 0.4       ])

In [28]:
from sklearn.preprocessing import StandardScaler  # doctest: +SKIP
scaler = StandardScaler()  # doctest: +SKIP
# Don't cheat - fit only on training data
scaler.fit(X_train)  # doctest: +SKIP
X_train = scaler.transform(X_train)  # doctest: +SKIP
# apply same transformation to test data
X_test = scaler.transform(X_test)  # doctest: +SKIP

In [29]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X, y)

cross_val_score(clf, X_train,y_train, cv=10)


C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

array([1.        , 1.        , 0.92307692, 0.91666667, 1.        ,
       0.91666667, 0.91666667, 1.        , 1.        , 0.9       ])

In [31]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, y_train)
cross_val_score(clf, X_train,y_train, cv=10)


C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\zhy89\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

array([1.        , 1.        , 0.92307692, 0.91666667, 1.        ,
       0.91666667, 0.91666667, 1.        , 1.        , 0.9       ])

In [34]:
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train, y_train)
cross_val_score(lin_clf, X_train,y_train, cv=10)


array([1.        , 0.92307692, 1.        , 0.91666667, 1.        ,
       0.91666667, 1.        , 1.        , 1.        , 1.        ])